<a href="https://colab.research.google.com/github/ennergarcia/customer_segmentation/blob/main/Segmentacao_clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color='white'>Linguagem Python no Jupyter Notebook</font>

Problema:

Considerando dados de clientes e seus respectivos perfis, irei realizar um agrupamento (segmentação) dos clientes por similaridade de características em n grupos.

In [4]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.12.12


In [5]:
# Imports
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [10]:
# Carrega os dados
df_custseg = pd.read_csv('https://raw.githubusercontent.com/ennergarcia/customer_segmentation/main/Clientes.csv')

In [11]:
# Observe que trata-se de uma dataframe pandas
type(df_custseg)

pandas.core.frame.DataFrame

In [12]:
# Visualiza as 10 primeiras linhas
df_custseg.head(10)

,ID_Cliente,Idade,Peso,Altura,Estado Civil,Cidade,Estado,Limite de Crédito,Taxa de Desconto,Tipo de Cliente
0,1,34.0,98.0,184,Casado,São Paulo,RS,581,6,Diamante
1,2,28.0,105.0,184,Casado,São Paulo,RJ,4537,4,Ouro
2,3,NaN,44.0,178,Divorciado,São Paulo,RJ,2977,5,Prata
3,4,33.0,84.0,164,Divorciado,São Paulo,SP,7462,3,Prata
4,5,56.0,NaN,189,Divorciado,Rio de Janeiro,RJ,3766,3,Bronze
5,6,28.0,80.0,173,Casado,Rio de Janeiro,SP,8173,5,Ouro
6,7,53.0,74.0,177,Casado,Belo Horizonte,MG,4020,5,Prata
7,8,32.0,NaN,179,Solteiro,Porto Alegre,MG,4534,2,Ouro
8,9,18.0,91.0,185,Casado,Rio de Janeiro,RS,4956,6,Bronze
9,10,38.0,47.0,270,Casado,Belo Horizonte,RJ,6102,4,Prata


In [13]:
# Sabemos que colunas com nomes contendo espaços, letras maiusculas e acentos podem se tornar um inconveniente, portanto, quanto antes as tratarmos, melhor.
df_custseg.rename(columns={'ID_Cliente': 'id_cliente', 'Idade': 'idade', 'Peso': 'peso', 'Altura': 'altura', 'Estado Civil': 'estado_civil', 'Cidade': 'cidade', 'Estado': 'estado', 'Limite de Crédito': 'limite_credito', 'Taxa de Desconto': 'taxa_desconto', 'Tipo de Cliente': 'tipo_cliente'}, inplace=True)

In [14]:
# Visualizando os novos nomes das colunas
df_custseg.head(10)

,id_cliente,idade,peso,altura,estado_civil,cidade,estado,limite_credito,taxa_desconto,tipo_cliente
0,1,34.0,98.0,184,Casado,São Paulo,RS,581,6,Diamante
1,2,28.0,105.0,184,Casado,São Paulo,RJ,4537,4,Ouro
2,3,NaN,44.0,178,Divorciado,São Paulo,RJ,2977,5,Prata
3,4,33.0,84.0,164,Divorciado,São Paulo,SP,7462,3,Prata
4,5,56.0,NaN,189,Divorciado,Rio de Janeiro,RJ,3766,3,Bronze
5,6,28.0,80.0,173,Casado,Rio de Janeiro,SP,8173,5,Ouro
6,7,53.0,74.0,177,Casado,Belo Horizonte,MG,4020,5,Prata
7,8,32.0,NaN,179,Solteiro,Porto Alegre,MG,4534,2,Ouro
8,9,18.0,91.0,185,Casado,Rio de Janeiro,RS,4956,6,Bronze
9,10,38.0,47.0,270,Casado,Belo Horizonte,RJ,6102,4,Prata


In [15]:
# Resumo estatístico
df_custseg[['idade', 'peso', 'altura','limite_credito']].describe()

,idade,peso,altura,limite_credito
count,498.000000,498.000000,502.000000,502.000000
mean,39.463855,69.913655,173.599602,5055.948207
std,9.984278,14.888539,12.518451,1932.759919
min,12.000000,27.000000,146.000000,-307.000000
25%,33.000000,60.000000,167.000000,3650.750000
50%,40.000000,70.000000,172.000000,5037.500000
75%,46.000000,80.000000,180.000000,6429.750000
max,68.000000,112.000000,278.000000,11853.000000


## Ponto de atenção

Observe que os itens de idade e peso tem menos itens na contagem o que demonstra que existem dados nulos (vazios) e estes devem ser tratados antes de seguir.